# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd #Импортируем библиотеку pandas и присваиваем ей псевдоним pd

df = pd.read_csv('/datasets/data.csv') #Сохранили датасет в переменной df

df.info() #Посмотрели информацию о таблице, увидели, что в столбцах 'days_employed' и 'total_income' имеются пропуски.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df['days_employed'].isna().sum() # Посчитали количество пропусков в столбце 'days_employed'

2174

In [3]:
df['total_income'].isna().sum() # Посчитали количество пропусков в столбце 'total_income'

2174

In [4]:
df.head(10) # Ознакомились с первыми десятью строками таблицы, обратили внимание на то, что значения столбца
            # 'days_employed' представлены в странном виде: отрицательные, либо слишком большие значения.
            # Значения столбца "education" имеют дубликаты, написанные в разных регистрах букв.

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

### Шаг 2.1 Заполнение пропусков

In [5]:
# Выясним, какую долю составляют пропущенные значения в столбцах 'days_employed' и 'total_income'
# Результат выведем в процентах с точностью до одного знака после запятой

skipped_part_de = df['days_employed'].isna().sum() / df['days_employed'].shape[0]
skipped_part_ti = df['total_income'].isna().sum() / df['total_income'].shape[0]

print("Доля пропущенных значений в столбце 'days_employed': {:.1%}".format(skipped_part_de))
print("Доля пропущенных значений в столбце 'total_income': {:.1%}".format(skipped_part_ti))

# Доли пропущенных значений равны, можно предположить, что пропуски обоих столбцов возникают в одних и тех же строках.

Доля пропущенных значений в столбце 'days_employed': 10.1%
Доля пропущенных значений в столбце 'total_income': 10.1%


In [6]:
# Создадим таблицу skipped_values_df, которая будет хранить строки с пропущенными значениями в обоих столбцах одновременно

skipped_values_df = df[(df['total_income'].isna()) & (df['days_employed'].isna())]

# Если количество строк этой таблицы будет равно количеству пропусков в каждом из столбцов, а мы помним, что 
# количество пропусков в обоих столбцах равно, то можно утверждать, что пропуски возникают в одних и тех же строках,
# то есть ошибка систематическая. Проверим, так ли это на самом деле.

nans_counter = df['days_employed'].isna().sum() # Для удобства сохраним количество пропусков в переменной nans_counter

if skipped_values_df.shape[0] == nans_counter:
    print('Предположение о пропусках в одних и тех же строках оправдано:')
    print('Пропуски возникают в одих и тех же строках в обоих столбцах сразу.')
    print('Возможная причина появления: ошибка при выгрузке данных, технологическая ошибка.')
else:
    print('Пропуски возникают в разных строках.')

Предположение о пропусках в одних и тех же строках оправдано:
Пропуски возникают в одих и тех же строках в обоих столбцах сразу.
Возможная причина появления: ошибка при выгрузке данных, технологическая ошибка.


In [7]:
# Так как столбцы 'days_employed' и 'total_income' содержат значения количественных переменных,
# а тип данных в этих столбцах числовой (float64) — заменить пропуски в столбцах мы можем медианными значениями без преобразований

# Почему медиана: Среднее значение некорректно характеризует данные, когда некоторые значения сильно 
# выделяются среди большинства. К тому же, столбец 'total_income' содержит значения дохода заёмщиков, усреднять эти значения
# было бы неверно. 

days_employed_med = df['days_employed'].median() #Посчитали медианное значение для столбца 'days_employed'
total_income_med = df['total_income'].median() #Посчитали медианное значение для столбца 'total_income'

df['days_employed'] = df['days_employed'].fillna(days_employed_med) # Заменили пропуски в столбце 'days_employed'
df['total_income'] = df['total_income'].fillna(total_income_med) # Заменили пропуски в столбце 'total_income'

In [8]:
df['days_employed'].isna().sum() # Посчитали количество пропусков в столбце 'days_employed', убедились, что их нет

0

In [9]:
df['total_income'].isna().sum() # Посчитали количество пропусков в столбце 'total_income', убедились, что их нет

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [10]:
# Обратим внимание на отрицательные значения в столбце 'days_employed', а также на слишком большие значения этого столбца
# Очевидно, что значение 340266.072047 хоть и является положительным, но не может соответствовать действительности

# Сначала выясним долю отрицательных значений столбца 'days_employed'

minus_values_df = df[df['days_employed'] < 0] #получили строки из таблицы df только с отрицательными значениями и сохранили их
minus_values_df.shape[0] # выяснили, что строк с отрицательными значениями 18080

18080

In [11]:
minus_values_part = minus_values_df.shape[0] / df.shape[0] #отношение строк с отрицательными значениями ко всей таблице df
print("Доля отрицательных значений столбца 'days_employed': {:.1%}".format(minus_values_part))

Доля отрицательных значений столбца 'days_employed': 84.0%


In [12]:
# 84% — это очень большая доля "аномалий", можно предположить, что минус в этих значениях оказался случайно.
# Отсортируем таблицу с отрицательными значениями по возрастанию значений столбца 'days_employed', чтобы ознакомиться
# с минимальными значениями и оценить, насколько они имели бы место быть, если бы являлись положительными.

minus_values_df.sort_values('days_employed').head(10)

# Начиная с первой строки видим аномальные значения: если представить, что наши значения положительные, то заемщику
# с минимальным значением в столбце 'days_employed' в возрасте 61 года нужно было бы работать около 50 лет 
# для достижения значения в 18388 дней, а начало трудовой деятельности в 11 лет сомнительно.

# Какие выводы можно сделать сейчас: 
# 1) отрицательные значения на самом деле можно перевести в положительные, скорее всего, минус закрался в данные случайно.
# 2) после перевода отрицательных значений в положительные будем считать за аномалии значения дней, которые больше
# количества дней, прошедших у заёмщика с наступления трудоспособного возраста.
# Нижняя граница трудоспособного возраста — 16 лет у представителей обоих полов. Источник — быстрый ответ Яндекс.поиска.

children  days_employed  dob_years education  education_id  \
16335         1  -18388.949901         61   среднее             1   
4299          0  -17615.563266         61   среднее             1   
7329          0  -16593.472817         60    высшее             0   
17838         0  -16264.699501         59   среднее             1   
16825         0  -16119.687737         64   среднее             1   
3974          0  -15835.725775         64   среднее             1   
1539          0  -15785.678893         59    высшее             0   
4321          0  -15773.061335         61   среднее             1   
7731          0  -15618.063786         64   среднее             1   
15675         0  -15410.040779         65    высшее             0   

               family_status  family_status_id gender income_type  debt  \
16335        женат / замужем                 0      F   сотрудник     0   
4299         женат / замужем                 0      F   компаньон     0   
7329         женат / замужем                 0      F   сотрудник     0   
17838        женат / замужем                 0      F   сотрудник     0   
16825        женат / замужем                 0      F   сотрудник     0   
3974        гражданский брак                 1      F   компаньон     0   
1539   Не женат / не замужем                 4      F   сотрудник     0   
4321        гражданский брак                 1      F   сотрудник     0   
7731         женат / замужем                 0      F   компаньон     0   
15675        женат / замужем                 0      F   сотрудник     0   

        total_income                                purpose  
16335  186178.934089               операции с недвижимостью  
4299   122560.741753                          покупка жилья  
7329   124697.846781           заняться высшим образованием  
17838   51238.967133                  на покупку автомобиля  
16825   91527.685995             покупка жилой недвижимости  
3974    96858.531436                        сыграть свадьбу  
1539   119563.851852  операции с коммерческой недвижимостью  
4321   205868.585780                                свадьба  
7731   296525.358574                     высшее образование  
15675  188800.068859             покупка жилой недвижимости

In [13]:
# Создадим функцию для перевода отрицательных значений дней в положительные
def module_days(days):
    if days < 0: # Если число дней отрицательно
        return abs(days) # Функция вернёт модуль числа
    return days # Иначе функция вернёт число дней без изменений


df['days_employed'] = df['days_employed'].apply(module_days) # Применим написанную функцию к столбцу 'days_employed'


df.head(10) # Убедимся в том, что изменения произошли

children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   СРЕДНЕЕ             1   
8         2    6929.865299         35    ВЫСШЕЕ             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

In [14]:
# К правке аномалий в столбце 'days_employed' вернёмся чуть позже, пока определим и исправим аномалии в других столбцах
# Начнём со столбца 'children'

df['children'].value_counts()

# Здесь внимание привлекают значения -1 и 20. Очевидно, что -1 ребёнок в семье — это некорректное значение и, скорее всего,
# знак минуса оказался в значении случайно. Так же привлекает внимание значение 20. До сих пор мы рассматривали количество детей
# как количественную переменную, но вот этот выброс в виде 20 (если я не прав, пожалуйста, поправьте меня!)
# заставляет задуматься о том, чтобы значение "20" заменить на "6" и после такой замены относиться к значениям столбца 'children'
# как к категориальным, или, если точнее, к ранговым. Мы имеем достаточно большое количество наблюдений и значение 20 уж очень 
# сильно отклоняется, это явный выход за две сигмы. Под рангом "6" будем иметь в виду число детей 6+ — это объяснит факт, что
# число заёмщиков с числом детей 6+ (6, 7, 8 и т д) больше, чем число заёмщиков с числом детей равным четырём и пяти вместе взятым.

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [15]:
# Создадим функцию для перевода отрицательных значений в положительные и замены числа 20 на число 6:
def change_children(children):
    if children == -1:
        return abs(children)
    elif children == 20:
        return 6
    return children

df['children'] = df['children'].apply(change_children) # Применим написанную функцию к столбцу 'children'

df['children'].value_counts() # Убедимся, что изменения вступили в силу

0    14149
1     4865
2     2055
3      330
6       76
4       41
5        9
Name: children, dtype: int64

In [16]:
# Исследуем столбец 'dob_years' на аномалии
df['dob_years'].value_counts()

# В значениях присутствует ноль. Это аномалия. В случае возраста считать среднее правильнее, поэтому, исключив аномалии,
# рассчитаем среднее и заменим нулевые значения

avg_age = df[df['dob_years'] != 0]['dob_years'].mean() # Рассчитываем средний возраст заёмщика

avg_age = int(avg_age) # Столбец хранит значения целого типа, приведём значение среднего возраста тоже к целому типу

print('Средний возраст заёмщика:', avg_age)

Средний возраст заёмщика: 43


In [17]:
df.loc[df['dob_years'] == 0, 'dob_years'] = avg_age # Заменили нули в столбце на значение среднего возраста

df['dob_years'].value_counts() # Убедились, что изменения вступили в силу

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [18]:
# Проверим столбец 'education' на наличие аномалий.
df['education'].value_counts()

# Аномалий нет, есть дубликаты, с ними будем разбираться в соответствующем разделе проекта :)

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [19]:
# Проверим столбец 'education' на наличие аномалий.

df['education_id'].value_counts()
# Аномалий нет.

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [20]:
# Из оставшихся столбцов проверим столбцы 'gender' и 'total_income', а после вернёмся к столбцу 'days_employed'

df['gender'].value_counts() # Проверяем значения столбца 'gender'


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [21]:
# Количество значений 'F' превосходит количетсво значений 'M' (другого значения в этом столбце быть не может), заменим
# 'XNA' на 'F'

df.loc[df['gender'] == 'XNA', 'gender'] = 'F'

df['gender'].value_counts() # Убедимся, что изменения вступили в силу

F    14237
M     7288
Name: gender, dtype: int64

In [22]:
df[df['total_income'] < 0]['total_income'].count() # Проверяем, что среди значений столбца 'total_income' нет отрицательных

0

In [23]:
# Вернёмся к аномалиям в столбце 'days_employed'. Количество дней на работе не может превышать то количество дней, которое
# прошло у заёмщика с наступления трудоспособного возраста. Вычислим среднее значение столбца без аномалий.
# В случае прошедшего времени лучше вычислять среднее значение.
# Получим таблицу без аномалий в столбце 'days_employed'

df_without_anomalies = df[df['days_employed'] <= (df['dob_years'] - 16) * 365]

# В такую таблицу отберём те строки таблицы df, у которых значение столбца 'days_employed' меньше, либо равно 
# произведению разности значения возраста заёмщика и нижней границы трудоспособного возраста и числа дней в году.

df_without_anomalies.head(10) # Посмотрим на такую таблицу

children  days_employed  dob_years education  education_id  \
0          1    8437.673028         42    высшее             0   
1          1    4024.803754         36   среднее             1   
2          0    5623.422610         33   Среднее             1   
3          3    4124.747207         32   среднее             1   
5          0     926.185831         27    высшее             0   
6          0    2879.202052         43    высшее             0   
7          0     152.779569         50   СРЕДНЕЕ             1   
8          2    6929.865299         35    ВЫСШЕЕ             0   
9          0    2188.756445         41   среднее             1   
10         2    4171.483647         36    высшее             0   

       family_status  family_status_id gender income_type  debt  \
0    женат / замужем                 0      F   сотрудник     0   
1    женат / замужем                 0      F   сотрудник     0   
2    женат / замужем                 0      M   сотрудник     0   
3    женат / замужем                 0      M   сотрудник     0   
5   гражданский брак                 1      M   компаньон     0   
6    женат / замужем                 0      F   компаньон     0   
7    женат / замужем                 0      M   сотрудник     0   
8   гражданский брак                 1      F   сотрудник     0   
9    женат / замужем                 0      M   сотрудник     0   
10   женат / замужем                 0      M   компаньон     0   

     total_income                     purpose  
0   253875.639453               покупка жилья  
1   112080.014102     приобретение автомобиля  
2   145885.952297               покупка жилья  
3   267628.550329  дополнительное образование  
5   255763.565419               покупка жилья  
6   240525.971920           операции с жильем  
7   135823.934197                 образование  
8    95856.832424       на проведение свадьбы  
9   144425.938277     покупка жилья для семьи  
10  113943.491460        покупка недвижимости

In [24]:
# Рассчитаем долю аномалий в столбце 'days_employed', прежде, чем будем вычислять среднее значение

anomalies_counter =  df.shape[0] - df_without_anomalies.shape[0] 
print("Количество аномалий в столбце 'days_employed':", anomalies_counter)

part_anomalies = anomalies_counter / df.shape[0]
print("Что составляет {:.1%}".format(part_anomalies), 'от общего числа наблюдений.')

Количество аномалий в столбце 'days_employed': 3537
Что составляет 16.4% от общего числа наблюдений.


In [25]:
# Теперь разобьём заёмщиков из таблицы без аномалий на категории. За ключевой фактор, влияющий на продолжительность количества
# дней проведённых на работе возьмём возраст заёмщиков. Для определения возрастных групп рассчитаем такие показатели, как
# медиана и квартили — показателя будет три, группы получим четыре. В рассчётах будем использовать метод median() 
# и логическую индексацию.

median_age = int(df_without_anomalies['dob_years'].median()) #столбец имеет тип int64, получим значение такого же типа

print('Медианный возраст заёмщика:', median_age)

# Рассчитаем значение первого квартиля
first_quartile_age = int(df_without_anomalies[df_without_anomalies['dob_years'] < median_age]['dob_years'].median())
print('Значение первого квартиля:', first_quartile_age)

# Рассчитаем значение третьего квартиля
third_quartile_age = int(df_without_anomalies[df_without_anomalies['dob_years'] > median_age]['dob_years'].median())
print('Значение третьего квартиля:', third_quartile_age)


# Мы получили четыре группы:

# 1) До 32
# 2) От 32 до 40
# 3) От 40 до 49
# 4) От 49

# В каждой из групп сосредоточено примерно одинаковое количество заёмщиков. Для каждой из таких групп расчитаем среднее значение
# по столбцу 'days_employed'

Медианный возраст заёмщика: 40
Значение первого квартиля: 32
Значение третьего квартиля: 49


In [26]:
# Считаем средние значения для каждой возрастной группы:

first_age_group_mean = df_without_anomalies[df_without_anomalies['dob_years'] < first_quartile_age]['days_employed'].mean()
second_age_group_mean = df_without_anomalies[(df_without_anomalies['dob_years'] >= first_quartile_age) & (df_without_anomalies['dob_years'] < median_age)]['days_employed'].mean()
third_age_group_mean = df_without_anomalies[(df_without_anomalies['dob_years'] >= median_age) & (df_without_anomalies['dob_years'] < third_quartile_age)]['days_employed'].mean()
fourth_age_group_mean = df_without_anomalies[df_without_anomalies['dob_years'] >= third_quartile_age]['days_employed'].mean()

print("Среднее значение столбца 'days_employed' для первой возрастной группы:", first_age_group_mean)
print("Среднее значение столбца 'days_employed' для второй возрастной группы:", second_age_group_mean)
print("Среднее значение столбца 'days_employed' для третьей возрастной группы:", third_age_group_mean)
print("Среднее значение столбца 'days_employed' для четвёртой возрастной группы:", fourth_age_group_mean)


Среднее значение столбца 'days_employed' для первой возрастной группы: 1302.9107325303262
Среднее значение столбца 'days_employed' для второй возрастной группы: 1983.626195368325
Среднее значение столбца 'days_employed' для третьей возрастной группы: 2493.702886689067
Среднее значение столбца 'days_employed' для четвёртой возрастной группы: 2885.6740628590856


In [27]:
# И действительно, мы можем наблюдать рост среднего числа дней, проведённых на рабочем месте с ростом возраста заёмщиков.
# Также перестрахуемся от возможной ошибки: мы не можем присвоить среднее значение столбца для первой возрастной группы 
# лицам моложе 19,5 лет (округлим до 20), так как количество дней (1302) будет больше возможных, если считать трудодни
# таких заёмщиков с 16 лет, поэтому создадим дополнительную группу zero_age_group_mean для девятнадцатилетних.
# 16-17-18 летних-заёмщиков в таблице нет, в чём не сложно убедиться

sixteens_counter = df[df['dob_years'] == 16]['dob_years'].count()
seventeens_counter = df[df['dob_years'] == 17]['dob_years'].count()
eighteens_counter = df[df['dob_years'] == 18]['dob_years'].count()
nineteens_counter = df[df['dob_years'] == 19]['dob_years'].count()

print('Число 16-летних заёмщиков:', sixteens_counter)
print('Число 17-летних заёмщиков:', seventeens_counter)
print('Число 18-летних заёмщиков:', eighteens_counter)
print('Число 19-летних заёмщиков:', nineteens_counter) # Уже 14 заёмщиков в возрасте 19 лет, для них мы и посчитаем среднее
print()

# В рассчётах среднего будем основываться на данных из таблицы без аномалий в столбце'days_employed'
zero_age_group_mean = df_without_anomalies[df_without_anomalies['dob_years'] == 19]['days_employed'].mean()
print("Среднее значение столбца 'days_employed' для девятнадцатилетних заёмщиков:", zero_age_group_mean)

Число 16-летних заёмщиков: 0
Число 17-летних заёмщиков: 0
Число 18-летних заёмщиков: 0
Число 19-летних заёмщиков: 14

Среднее значение столбца 'days_employed' для девятнадцатилетних заёмщиков: 633.6780859334784


In [28]:
# Теперь заменим аномалии в каждой из групп на среднее значение группы.

# Напишем функцию change_employed_days()

def change_employed_days(row):
    if row['days_employed'] > (row['dob_years'] - 16) * 365: # Если значение дней с момента трудоустройства больше возможных
        if row['dob_years'] <= 19: # если значение возраста меньше либо равно 19
            return zero_age_group_mean # функция вернёт среднее значение нулевой группы, рассчитанное для 19-летних
        elif row['dob_years'] > 19 and row['dob_years'] < first_quartile_age: # если возраст от 20 до значения первого квартиля
            return first_age_group_mean # функция вернёт среднее значение первой возрастной группы
        elif row['dob_years'] >= first_quartile_age and row['dob_years'] < median_age: # если возраст в диапазоне от первого квартиля до второго
            return second_age_group_mean # функция вернёт среднее значение второй возрастной группы 
        elif row['dob_years'] >= median_age and row['dob_years'] < third_quartile_age: # если возраст в диапазоне от второго до третьего квартиля
            return third_age_group_mean # функция вернёт среднее значение третьей возрастной группы 
        elif row['dob_years'] >= third_quartile_age: # если значение возраста за третьим квартилем
            return fourth_age_group_mean # функция вернёт среднее значение четвёртой возрастной группы
    return row['days_employed'] # иначе (если значение дней с трудоустройства не больше возможных) вернём значение без изменений



df['days_employed'] = df.apply(change_employed_days, axis=1) #вызовем функцию change_employed_days() для столбца 'days_employed'


# Убедимся, что теперь таблица не содержит аномальных значений в столбце days_employed'

df[df['days_employed'] > (df['dob_years'] - 16) * 365]['days_employed'].count()


# Аномальных значений больше не имеем

0

# Шаг 2.3. Изменение типов данных.

In [29]:
# Заменим вещественный тип данных в столбце 'total_income' на целочисленный с помощью метода astype()

df['total_income'] = df['total_income'].astype('int')

# Убедимся, что тип данных изменён успешно

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [30]:
# Также заменим тип данных в столбце 'days_employed' — количество дней в дробном варианте выглядит весьма подозрительно.

df['days_employed'] = df['days_employed'].astype('int')

# Убедимся, что тип данных изменён успешно

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [31]:
# Проверим таблицу на наличие явных дубликатов с помощью связки методов duplicated() и sum()

df.duplicated().sum()

54

In [32]:
# Удалим явные дубликаты и переприсвоим индексы для таблицы без дубликатов

df = df.drop_duplicates().reset_index(drop=True)

# Убедимся, что явных дубликатов больше нет

df.duplicated().sum()

0

In [33]:
# Теперь перейдём к неявным дубликатам. Столбец 'education' содержит дубликаты в разном регистре

df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [34]:
df['education'] = df['education'].str.lower() # Используем метод str.lower() для приведения значений столдца к одному регистру
df['education'].value_counts() # Убедимся, что изменения вступили в силу

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [35]:
# Проверим наличие дубликатов в столбце 'family_status'
df['family_status'].value_counts()

# Дубликатов нет

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [36]:
# Проверим наличие дубликатов в столбце 'income_type'
df['income_type'].value_counts()

# Дубликатов нет
# Значениями столбца 'purpose' займёмся в одном из последних пунктов проекта




сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Почему был выбрана связка duplicated() и sum() для поиска дубликатов: вызов метода duplicated() у датафрейма возращает логические значения, а метод sum() их подсчитывает и в результате применения этих методов мы получаем число, которое сообщает нам о количестве явных дубликатов. Если число больше нуля, явные дубликаты есть, иначе — отсутствуют.

Почему была выбрана связка методов drop_duplicates() и reset_index() с аргументом drop в значении True:
Метод drop_duplicates() удаляет из таблицы явные дубликаты, а reset_index с аргументом drop=True переназначает новые индексы,
удаляя старые.

Для поиска неявных дубликатов в столбце 'education' был использован метод value_counts, который показывал значения столбца
и частоту их использования. Так же можно было бы использовать метод unique(), чтобы увидеть уникальные значения столбца без
их количетсва, но value_counts() более подробный. С его помощью можно оценить визуально наличие дубликатов, написанных в
разных регистрах.

Для приведения значений столбца 'education' к единому регистру был использован метод str.lower(). Его использование удобно тем,
Что нам не нужно писать дополнительную функцию — примение метода к столбцу с его последующим сохранением в столбце приводит
значения этого столбца к нижнему регистру.

Каким образом могли появиться дубликаты: вероятно, данные о заёмщиках могли поступить из разных источников и при их
"объединении" в одном датасете могли оказаться, как дублирующие друг друга строки, так и значения, написанные в разных регистрах.
По количеству разных вариантов написания можно сделать предположение о количестве источников, из которых в основной датасет
поступали данные. Также можно предполагать человеческий фактор — ошибку сотрудника, который отвечал за составление датасета.
Явных дубликатов было не много: 54 записи из 21525 строк. Это 0,2% от всех данных. Человек ошибается, это факт.
Ещё возможно предположение о том, что не был изначально согласован формат записи данных в столбце 'education'.
Совершенно очевидно, что согласования значения для столбца 'purpose' так же не происходило, но к нему мы ещё придём.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [37]:
# Создадим два новых датафрейма: df_education и df_family_status, они станут "словарями" и мы сможем обращаться к ним по id.

df_education = df[['education_id', 'education']] 
df_family_status = df[['family_status_id', 'family_status']]

In [38]:
# Посмотрим на первые 10 строк "словаря" df_education

df_education.head(10)

education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
5             0    высшее
6             0    высшее
7             1   среднее
8             0    высшее
9             1   среднее

In [39]:
# Посмотрим на первые 10 строк "словаря" df_family_status

df_family_status.head(10)

family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем

In [40]:
# Совершенно очевидно, что наши "словари" содержат множество явных дубликатов, от которых надо избавиться. 
# Сделаем это при помощи связки методов drop_duplicates() и reset_index()

df_education = df_education.drop_duplicates().reset_index(drop=True)
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True)

# Убедимся, что дубликатов нет

print("Количество явных дубликатов в таблице 'df_education':", df_education.duplicated().sum())
print("Количество явных дубликатов в таблице 'df_family_status':", df_family_status.duplicated().sum())

Количество явных дубликатов в таблице 'df_education': 0
Количество явных дубликатов в таблице 'df_family_status': 0


In [41]:
# Теперь удалим из исходной таблицы столбцы 'education' и 'family_status'

df = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]

df.head(10) # Убедимся, что удаление прошло успешно

children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0           2885         53             1                 1      F   
5         0            926         27             0                 1      M   
6         0           2879         43             0                 0      F   
7         0            152         50             1                 0      M   
8         2           6929         35             0                 1      F   
9         0           2188         41             1                 0      M   

  income_type  debt  total_income                     purpose  
0   сотрудник     0        253875               покупка жилья  
1   сотрудник     0        112080     приобретение автомобиля  
2   сотрудник     0        145885               покупка жилья  
3   сотрудник     0        267628  дополнительное образование  
4   пенсионер     0        158616             сыграть свадьбу  
5   компаньон     0        255763               покупка жилья  
6   компаньон     0        240525           операции с жильем  
7   сотрудник     0        135823                 образование  
8   сотрудник     0         95856       на проведение свадьбы  
9   сотрудник     0        144425     покупка жилья для семьи

### Шаг 2.6. Категоризация дохода.

In [42]:
# На основании диапазонов:

# 0–30000 — 'E'
# 30001–50000 — 'D'
# 50001–200000 — 'C'
# 200001–1000000 — 'B'
# 1000001 и выше — 'A'

# Создадим столбец 'total_income_category' с категориями. 
# Для этого напишем функцию и применим её с помощью метода apply()

def make_category(income):
    if income <= 30000:
        return 'E'
    elif income <= 50000:
        return 'D'
    elif income <= 200000:
        return 'C'
    elif income <= 1000000:
        return 'B'
    else:
        return 'A'
    
df['total_income_category'] = df['total_income'].apply(make_category) # Создаём столбец 'total_income_category'
df.head(10) # Убедимся, что столбец добавлен в таблицу

children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0           2885         53             1                 1      F   
5         0            926         27             0                 1      M   
6         0           2879         43             0                 0      F   
7         0            152         50             1                 0      M   
8         2           6929         35             0                 1      F   
9         0           2188         41             1                 0      M   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупка жилья   
1   сотрудник     0        112080     приобретение автомобиля   
2   сотрудник     0        145885               покупка жилья   
3   сотрудник     0        267628  дополнительное образование   
4   пенсионер     0        158616             сыграть свадьбу   
5   компаньон     0        255763               покупка жилья   
6   компаньон     0        240525           операции с жильем   
7   сотрудник     0        135823                 образование   
8   сотрудник     0         95856       на проведение свадьбы   
9   сотрудник     0        144425     покупка жилья для семьи   

  total_income_category  
0                     B  
1                     C  
2                     C  
3                     B  
4                     C  
5                     B  
6                     B  
7                     C  
8                     C  
9                     C

### Шаг 2.7. Категоризация целей кредита.

In [43]:
# Теперь создадим функцию, которая на основании данных из столбца 'purpose' сформирует новый столбец 'purpose_category', 
# в который войдут следующие категории:

# 'операции с автомобилем'
# 'операции с недвижимостью'
# 'проведение свадьбы'
# 'получение образования'

# Для начала изучим данные в столбце 'purpose' и определим, какие подстроки помогут правильно определить категорию.

df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [44]:
# для категории 'операции с недвижимостью' будем искать подстроку 'недвиж' или подстроку 'жил'
# для категории 'операции с автомобилем' будем искать подстроку 'автомобил'
# для категории 'проведение свадьбы' будем искать подстроку 'свадьб'
# для категории 'получение образования' будем искать подстроку 'образован'

def make_target(long_string):
    if 'автомобил' in long_string:
        return 'операции с автомобилем'
    if 'недвиж' in long_string or 'жил' in long_string:
        return 'операции с недвижимостью'
    if 'свадьб' in long_string:
        return 'проведение свадьбы'
    if 'образован' in long_string:
        return 'получение образования'
    
df['purpose_category'] = df['purpose'].apply(make_target) # Создаём столбец 'purpose_category'

df.head(10) # Проверяем, появился ли он в таблице

children  days_employed  dob_years  education_id  family_status_id gender  \
0         1           8437         42             0                 0      F   
1         1           4024         36             1                 0      F   
2         0           5623         33             1                 0      M   
3         3           4124         32             1                 0      M   
4         0           2885         53             1                 1      F   
5         0            926         27             0                 1      M   
6         0           2879         43             0                 0      F   
7         0            152         50             1                 0      M   
8         2           6929         35             0                 1      F   
9         0           2188         41             1                 0      M   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупка жилья   
1   сотрудник     0        112080     приобретение автомобиля   
2   сотрудник     0        145885               покупка жилья   
3   сотрудник     0        267628  дополнительное образование   
4   пенсионер     0        158616             сыграть свадьбу   
5   компаньон     0        255763               покупка жилья   
6   компаньон     0        240525           операции с жильем   
7   сотрудник     0        135823                 образование   
8   сотрудник     0         95856       на проведение свадьбы   
9   сотрудник     0        144425     покупка жилья для семьи   

  total_income_category          purpose_category  
0                     B  операции с недвижимостью  
1                     C    операции с автомобилем  
2                     C  операции с недвижимостью  
3                     B     получение образования  
4                     C        проведение свадьбы  
5                     B  операции с недвижимостью  
6                     B  операции с недвижимостью  
7                     C     получение образования  
8                     C        проведение свадьбы  
9                     C  операции с недвижимостью

In [45]:
# Также проверяем, сколько уникальных значений и в каком количестве содержится в нём
df['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [46]:
# Чтобы ответить на этот вопрос, построим свободную таблицу
# Для аргумента value будем использовать любой столбец, не содержащий нулевых значений, чтобы функцией count() посчитать
# число строк

first_pivot = df.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')
display(first_pivot)

debt            0       1
children                 
0         13044.0  1063.0
1          4411.0   445.0
2          1858.0   194.0
3           303.0    27.0
4            37.0     4.0
5             9.0     NaN
6            68.0     8.0

In [47]:
# Посчитаем долю тех заёмщиков, которые не возвращают кредит в срок для каждой из категорий

try:
    first_pivot['dept_part'] = first_pivot[1] / (first_pivot[0] + first_pivot[1])
    display(first_pivot)
except:
    print('Ошибка деления на ноль, проверьте данные')

debt            0       1  dept_part
children                            
0         13044.0  1063.0   0.075353
1          4411.0   445.0   0.091639
2          1858.0   194.0   0.094542
3           303.0    27.0   0.081818
4            37.0     4.0   0.097561
5             9.0     NaN        NaN
6            68.0     8.0   0.105263

Мы можем наблюдать отсутствие зависимости между количеством детей и возвратом кредита в срок, так как доля тех заёмщиков с детьми, которые не возвращают кредитные средства вовремя находится в диапазоне от 8 до 10% и не изменяется от группы к группе.

Отдельно можно отметить, что доля заемщиков, которые не возвращают кредитные средства вовремя чуть ниже среди заёмщиков без детей: она составляет 7,5 %.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
# Чтобы ответить на этот вопрос, так же построим свободную таблицу
second_pivot = df.pivot_table(index='family_status_id', columns='debt', values='gender', aggfunc='count')

try:
    second_pivot['dept_part'] = second_pivot[1] / (second_pivot[0] + second_pivot[1])
    display(second_pivot)
except:
    print('Ошибка деления на ноль, проверьте данные')

debt                  0    1  dept_part
family_status_id                       
0                 11413  931   0.075421
1                  3775  388   0.093202
2                   896   63   0.065693
3                  1110   85   0.071130
4                  2536  274   0.097509

Так же можем наблюдать отсутствие зависимости между семейным положением и возвратом кредита в срок — доли распределяются примерно одинаково (от 6,5% до 9,7%) вне зависимости от семейного положения. Самый низкий показатель 6,5% процентов принадлежит к группе заёмщиков с семейным положением вдова/вдовец.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
# Снова используем свобдную таблицу
third_pivot = df.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')

try:
    third_pivot['dept_part'] = third_pivot[1] / (third_pivot[0] + third_pivot[1])
    display(third_pivot)
except:
    print('Ошибка деления на ноль, проверьте данные')

debt                       0     1  dept_part
total_income_category                        
A                         23     2   0.080000
B                       4685   356   0.070621
C                      14673  1360   0.084825
D                        329    21   0.060000
E                         20     2   0.090909

Здесь мы наблюдаем отсутствие взаимосвязи. Зависимость между уровнем дохода и возвратом кредита в срок отсутствует.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [50]:
# Будем использовать свобдную таблицу для ответа на этот вопрос 
fourth_pivot = df.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')

try:
    fourth_pivot['dept_part'] = fourth_pivot[1] / (fourth_pivot[0] + fourth_pivot[1])
    display(fourth_pivot)
except:
    print('Ошибка деления на ноль, проверьте данные')

debt                          0    1  dept_part
purpose_category                               
операции с автомобилем     3905  403   0.093547
операции с недвижимостью  10032  782   0.072314
получение образования      3644  370   0.092177
проведение свадьбы         2149  186   0.079657

Чаще всего возврат кредита в срок происходит в категории "Операции с недвижимостью", по ней мы наблюдаем самую низкую долю задолженностей. 
Ниже всего показатели по возврату кредита в срок у категории "Операции с автомобилем" и "Получение образования", однако разрыв в значениях не настолько существенный, чтобы утверждать, что различия ощутимые.

## Общий вывод:

В результате проведённого исследования удалось установить отсутствие взаимосвязи между количеством детей у заёмщика, уровнем его образования, семейным положением и погашением задолженности по кредиту в срок. Доля заёмщиков, возвращающих средства в срок при исследовании такой взаимосвязи всегда была примерно одинаковой в сравнении по категориям. Отдельно можно отметить низкие значения по несвоевременному возврату кредитных средств у категорий заемщиков без детей, а также у категорий заёмщиков, семейное положение которых соответствует значению "вдовец/вдвова".

О несущественных различиях в доле заёмщиков, возвращающих кредиты в срок можно говорить в контексте цели кредита. Реже всего не укладываются в срок платежа заёмщики, которые брали кредит на операции с недвижимостью — чаще всего те, кто брал кредит на операции с автомобилем: 7,7% заёмщиков просрочили платежи против 10,3% заёмщиков соответственно.